# Usage in Python

- Python has support for working with different databases:
1. sqlite
2. mysql
3. postgres
4. etc.

## Psycopg
- Psycopg is the most popular PostgreSQL database adapter for python
- Psycopg 2 is mostly implemented in C
- Many Python types are supported and adapted to matching PostgreSQL data types

## Basic module usage

> python3 -m pip install psycopg2 

> create database python_db;

In [122]:
import psycopg2

#Connect to an existing database
conn = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)

#Open a cursor to perform database operations
cursor = conn.cursor()

#Execute a command: this creates a new table
cursor.execute("CREATE TABLE if not exists test (id serial PRIMARY KEY, num integer, data varchar);")

#Pass data to fill a query placeholders 
cursor.execute("INSERT INTO test (num, data) VALUES (%s, %s);", (100, "abc'def"))

# Query the database and obtain data as Python objects
cursor.execute("SELECT * FROM test;")
data = cursor.fetchone()

# Make the changes to the database persistent
conn.commit()

# Close communication with the database
#cursor.close()
#conn.close()
data

(1, 100, "abc'def")

In [123]:
cursor.execute("select * from test;")
cursor.fetchall()

[(1, 100, "abc'def"),
 (23, 22, 'BLA'),
 (2, 100, 'DATA'),
 (3, None, 'Bla'),
 (4, None, 'Bla'),
 (5, None, 'Bla'),
 (6, None, 'Bla'),
 (8, None, 'Bla'),
 (9, None, 'Bla'),
 (10, None, '("Bla'),
 (11, None, 'Bla'),
 (12, None, 'Bla'),
 (15, 1, None),
 (16, 1, None),
 (17, 1, None),
 (18, 1, None),
 (19, 1, None),
 (20, 5, None),
 (21, 10, None),
 (24, 100, "abc'def")]

- connect() creates a new database session
(a new connection instance is returned)

In [124]:
print(type(conn))

<class 'psycopg2.extensions.connection'>


- each connection/session can create new cursor instances
- cursor instances can execute database commands
- commands are sent by the following methods:
1. execute()
2. executemany()
- to retrieve data use following cursor methods:
1. fetchone() - gets one row of the query
2. fetchall() - gets all available rows
3. fetchmany() - gets as many rows as specified e.g. fetchmany(5)

### Access Data From a cursor
- You can have many cursors sharing the same connection to a database.


- a query can potentially match very large sets of data
- Read operations do not immediately return all values matching the query
- these operations rely on cursors
- cursors fetches data in batches to reduce memory consumption

- Closing the cursor frees resources associated to the queries.
- It would not eliminate the connection to the database itself.
(Therefore, there is no need for reauthentication)

- Cursors created from the same connection are not isolated
- i.e. any changes done to the database by a cursor are visible by other cursor


### Connections and Cursors
- Connections and cursors can be used as context managers and commits if no exception occurs. 
- the connection is not closed by the context
- the cursor is closed by the context

In [125]:
with conn:
    with conn.cursor() as curs:
        cursor.execute("CREATE TABLE if not exists test3 (id serial PRIMARY KEY, num integer, data varchar);")
        curs.execute("SELECT * FROM test;")
        curs.execute("SELECT count(*) FROM test;")
        print(curs.fetchall())
# curs.fetchall()

[(20,)]


### Passing parameters to SQL queries 
### execute and executemany()
- Execute a database operation 
- Parameters may be provided 
- For positional variables binding, the second argument must always be a sequence

In [126]:
with conn:
    with conn.cursor() as curs:
#INSERT INTO some_table (an_int, a_date, a_string)
#VALUES (21, 22, 'BLA');
        curs.execute("""
        insert into test (id, num, data)
        values (%s, %s, %s)
        """, (23, 22, 'BLA'))
        curs.execute("select * from test;")
        print(curs.fetchall())

UniqueViolation: duplicate key value violates unique constraint "test_pkey"
DETAIL:  Key (id)=(23) already exists.


- Named arguments are supported too using %(name)s placeholders in the query
- allows to specify the values in any order and to repeat the same value

In [ ]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%(value1)s, %(value2)s)
        """, {'value1':100, 'value2': 'DATA'})
        curs.execute("select * from test;")
        print(curs.fetchall())

[(1, 100, "abc'def"), (23, 22, 'BLA'), (2, 100, 'DATA')]


### Context and context object
- shows that its the same connection

In [ ]:
with conn as conn_val:
    with conn.cursor() as curs:
        curs.execute("select * from test;")
        print(curs.fetchall())
        print(conn_val)

[(1, 100, "abc'def"), (23, 22, 'BLA'), (2, 100, 'DATA')]
<connection object at 0x7fa506012e00; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>


In [ ]:
with conn as conn_val:
    with conn.cursor() as curs:
        curs.execute("select * from test;")
        print(curs.fetchall())
        print(conn_val)
print(conn_val)
# conn.close()
# print(conn_val)

[(1, 100, "abc'def"), (23, 22, 'BLA'), (2, 100, 'DATA')]
<connection object at 0x7fa506012e00; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>
<connection object at 0x7fa506012e00; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>


### The problem with the query parameters - SQL Injection

In [ ]:
with conn as conn_val:
    with conn.cursor() as curs:
        SQL = "INSERT INTO test (num) VALUES (%s);" # Note: no quotes, even without quotes
        user_input = ("1); select * from test where 1 in (1", ) #Malicious input
        curs.execute(SQL % user_input)
        print(curs.fetchall())

InvalidTextRepresentation: invalid input syntax for type integer: "1); select * from test where 1 in (1"
LINE 1: INSERT INTO test (num) VALUES ('1); select * from test where...
                                       ^


In [ ]:
with conn as conn_val:
    with conn.cursor() as curs:
        SQL = "INSERT INTO test (num) VALUES (%s);" # Note: no quotes, even without quotes
        user_input = ("1); select * from test where 1 in (1", ) #Malicious input
        curs.execute(SQL, user_input)# handling the quote; prevents injection
        print(curs.fetchall())

InvalidTextRepresentation: invalid input syntax for type integer: "1); select * from test where 1 in (1"
LINE 1: INSERT INTO test (num) VALUES ('1); select * from test where...
                                       ^


<span style=color:red;>Warning: Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. </span>

![](exploits_of_a_mom.png)

### executemany

In [127]:
with conn:
    with conn.cursor() as curs:
        nums = ((1,), (5,), (10,)) #tuple of tuple
        curs.executemany("INSERT INTO test (num) VALUES (%s)", nums)
        curs.execute("select * from test")
        print(curs.fetchall())


[(1, 100, "abc'def"), (23, 22, 'BLA'), (2, 100, 'DATA'), (3, None, 'Bla'), (4, None, 'Bla'), (5, None, 'Bla'), (6, None, 'Bla'), (8, None, 'Bla'), (9, None, 'Bla'), (10, None, '("Bla'), (11, None, 'Bla'), (12, None, 'Bla'), (15, 1, None), (16, 1, None), (17, 1, None), (18, 1, None), (19, 1, None), (20, 5, None), (21, 10, None), (24, 100, "abc'def"), (25, 1, None), (26, 5, None), (27, 10, None)]


- Parameters are bounded to the query using the same rules described in the execute() method.
- In its current implementation this method is not faster than executing execute() in a loop

### Adaptation of Python values to SQL types
- There is default mapping to convert Python types into PostgreSQL types, and vice versa

- None <-->NULL
- bool <-->bool
- decimal <--> numeric
- str <--> varchar
- float <--> real or double
- int <--> smallint or integer or bigint
- date <--> date
- time <--> timetz

#### mogrify
- Return a query string after arguments binding. 
- The string returned is exactly the one that would be sent by execute() method.


In [131]:
with conn:
    with conn.cursor() as curs:
        query1 = curs.mogrify("INSERT INTO test (num, data) VALUES (10, 'bal');")
        query2 = curs.mogrify("""
        INSERT INTO test (num, data) VALUES (%s, %s);
        """, (10,'BLA'))
        print(query2)

b"\n        INSERT INTO test (num, data) VALUES (10, 'BLA');\n        "


- the returned value is always a byte string

### Constants adaptation
- Python None, True and False are converted into the proper SQL literals:

In [136]:

with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(None, True, False))
        print(query)

b'\n        SELECT NULL, true, false;'


### Numeric adaptation

In [137]:
from decimal import Decimal
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(10, 10.00, Decimal("10.00")))
        print(query)

b'\n        SELECT 10, 10.0, 10.00;'


- sometimes you may prefer to receive numeric data as float instead:
- you can configure an adapter to cast PostgreSQL numeric to Python float.


### Date/Time objects adaptation

In [139]:
from datetime import datetime
dt = datetime.now()
dt
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(dt,dt.date(),dt.time()))
        print(query)


b"\n        SELECT '2023-03-09T01:06:00.642316'::timestamp, '2023-03-09'::date, '01:06:00.642316'::time;"


### Lists adaptation


In [143]:
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s;""",([10, 20, 30], ))
        print(query)

b'\n        select * from my_table where num in ARRAY[10,20,30];'
